In [1]:
import pandas_gbq

In [2]:
project_id = "kyd-storage"

In [3]:
df_eq = pandas_gbq.read_gbq(
    """
SELECT
  symbol,
  ed.refdate,
  case when ed.md_close is null and (ed.change_distribution_id = 0 or ed.change_distribution_id is null) then ed.ch_close
  else ed.md_close end close,
  case when ed.md_oscillation_percentage is null and (ed.change_distribution_id = 0 or ed.change_distribution_id is null) then ed.ch_oscillation_percentage
  else ed.md_oscillation_percentage end oscillation_percentage,
  ed.distribution_id,
  ed.change_distribution_id
FROM
  `kyd-storage.layer1_b3.tb_equities_aux` eq,
  UNNEST(equity_data) ed
WHERE
    symbol = 'BEEF3'
""",
    project_id,
)

Downloading: 100%|██████████| 1699/1699 [00:00<00:00, 3988.64rows/s]


In [5]:
df_eq

,symbol,refdate,close,oscillation_percentage,distribution_id,change_distribution_id
0,BEEF3,2016-01-04,12.50,NaN,105,<NA>
1,BEEF3,2016-01-05,12.94,3.51,105,0
2,BEEF3,2016-01-06,12.75,-1.46,105,0
3,BEEF3,2016-01-07,12.11,-5.01,105,0
4,BEEF3,2016-01-08,11.95,-1.32,105,0
...,...,...,...,...,...,...
1694,BEEF3,2022-11-07,14.03,-2.50,114,0
1695,BEEF3,2022-11-08,14.64,4.34,114,0
1696,BEEF3,2022-11-09,14.33,-2.11,114,0
1697,BEEF3,2022-11-10,13.73,-4.18,114,0


In [8]:
df_eq_ordered = df_eq.sort_values("refdate", ascending=False).reset_index(drop=True)
df_eq_ordered

,symbol,refdate,close,oscillation_percentage,distribution_id,change_distribution_id
0,BEEF3,2022-11-11,15.17,10.48,114,0
1,BEEF3,2022-11-10,13.73,-4.18,114,0
2,BEEF3,2022-11-09,14.33,-2.11,114,0
3,BEEF3,2022-11-08,14.64,4.34,114,0
4,BEEF3,2022-11-07,14.03,-2.50,114,0
...,...,...,...,...,...,...
1694,BEEF3,2016-01-08,11.95,-1.32,105,0
1695,BEEF3,2016-01-07,12.11,-5.01,105,0
1696,BEEF3,2016-01-06,12.75,-1.46,105,0
1697,BEEF3,2016-01-05,12.94,3.51,105,0


In [12]:
f = 1 + df_eq_ordered["oscillation_percentage"] / 100
f.cumprod(skipna=False).tail(20)

1679    1.558110
1680    1.606412
1681    1.635167
1682    1.656260
1683         NaN
1684         NaN
1685         NaN
1686         NaN
1687         NaN
1688         NaN
1689         NaN
1690         NaN
1691         NaN
1692         NaN
1693         NaN
1694         NaN
1695         NaN
1696         NaN
1697         NaN
1698         NaN
Name: oscillation_percentage, dtype: float64

In [ ]:
df_eq["refdate"] = df_eq["refdate"].astype(str)
df_eq = df_eq.sort_values("refdate", ascending=False).reset_index()
f = 1 + df_eq["oscillation_percentage"] / 100
f = f.cumprod()
f = f.shift()
f.iloc[0] = 1
df_eq["adj_close"] = df_eq.loc[0, "close"] / f